In [34]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# --- 1. Constants and Configuration ---

# This should match the MAX_SEQ_LEN from the Transformer model for consistency.
MAX_SEQ_LEN = 128

# --- 2. Data Loading and Preprocessing ---

def load_and_preprocess_data(proportions_file, ground_truth_file):
    """
    Loads predicted proportions, extracts statistical features, and aligns them with
    the true total time for each sequence.
    
    Args:
        proportions_file (str): Path to the CSV containing predicted proportions.
        ground_truth_file (str): Path to the original data file for true total time.

    Returns:
        A tuple containing:
        - Padded sequences of proportions (X_seq).
        - An array of step counts for each sequence (X_steps).
        - An array of statistical features for each sequence (X_stats).
        - An array of total times (y).
        - The dataframe from the proportions_file for final output generation.
    """
    if not os.path.exists(proportions_file):
        print(f"❌ Error: Proportions file not found at '{proportions_file}'")
        return None, None, None, None, None
    if not os.path.exists(ground_truth_file):
        print(f"❌ Error: Ground truth data file not found at '{ground_truth_file}'")
        return None, None, None, None, None

    props_df = pd.read_csv(proportions_file)
    truth_df = pd.read_csv(ground_truth_file)
    
    # --- Re-calculate the true total time using the definitive logic ---
    truth_df['step_duration'] = truth_df.groupby('SeqOrder')['timediff'].diff().fillna(truth_df['timediff'])
    truth_df['step_duration'] = truth_df['step_duration'].clip(lower=0)
    truth_df['Step'] = truth_df.groupby('SeqOrder').cumcount()
    
    end_marker_step = truth_df[truth_df['sourceID'] == 10].groupby('SeqOrder')['Step'].first()
    truth_df['end_marker_step'] = truth_df['SeqOrder'].map(end_marker_step)
    truth_df.loc[truth_df['Step'] > truth_df['end_marker_step'], 'step_duration'] = 0
    
    total_times = truth_df.groupby('SeqOrder')['step_duration'].sum()

    # --- Prepare data for the LSTM ---
    X_sequences, X_num_steps, X_stats = [], [], []
    
    for _, g in props_df.groupby('SeqOrder'):
        proportions = g['predicted_proportion'].values
        X_sequences.append(proportions.reshape(-1, 1))
        X_num_steps.append(len(g))
        
        # --- Feature Engineering: Create a richer set of statistical features ---
        stats = [
            np.mean(proportions),
            np.std(proportions),
            np.max(proportions),
            np.percentile(proportions, 25), # 25th percentile
            np.median(proportions),      # 50th percentile
            np.percentile(proportions, 75)  # 75th percentile
        ]
        X_stats.append(stats)
    
    y_total_times = props_df['SeqOrder'].unique()
    y_sequences = np.array([total_times.get(seq_id, 0) for seq_id in y_total_times])

    X_padded_seq = tf.keras.preprocessing.sequence.pad_sequences(
        X_sequences, maxlen=MAX_SEQ_LEN, padding='post', dtype='float32'
    )
    
    X_steps_arr = np.array(X_num_steps, dtype='float32').reshape(-1, 1)
    X_stats_arr = np.array(X_stats, dtype='float32')

    print(f"Successfully processed {len(X_padded_seq)} sequences.")
    
    return X_padded_seq, X_steps_arr, X_stats_arr, y_sequences.reshape(-1, 1), props_df


# --- 3. Residual Model Architecture with Attention ---

def build_residual_attention_model(sequence_shape, scalar_shape, stats_shape):
    """
    Builds a multi-input model with a residual architecture.
    """
    # --- Define All Inputs ---
    sequence_input = layers.Input(shape=sequence_shape, name='sequence_input')
    scalar_input = layers.Input(shape=scalar_shape, name='scalar_input')
    stats_input = layers.Input(shape=stats_shape, name='stats_input')

    # --- Branch 1: Baseline Model (using non-sequential features) ---
    baseline_features = layers.concatenate([scalar_input, stats_input])
    baseline_out = layers.Dense(32, activation='relu')(baseline_features)
    baseline_out = layers.Dense(16, activation='relu')(baseline_out)
    baseline_prediction = layers.Dense(1, name='baseline_prediction')(baseline_out)

    # --- Branch 2: Specialist Model (predicting the residual) ---
    masked_sequence = layers.Masking(mask_value=0.)(sequence_input)
    gru_out = layers.Bidirectional(layers.GRU(64, return_sequences=True))(masked_sequence)
    
    # Using MultiHeadAttention for robust self-attention
    # The output of Bidirectional GRU is 128 (64 forward + 64 backward)
    attention_out = layers.MultiHeadAttention(num_heads=4, key_dim=128)(query=gru_out, value=gru_out, key=gru_out)
    context_vector = layers.GlobalAveragePooling1D()(attention_out)
    
    # The specialist branch also gets the non-sequential features to have full context
    specialist_features = layers.concatenate([context_vector, scalar_input, stats_input])
    x = layers.Dense(64, activation='relu')(specialist_features)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(32, activation='relu')(x)
    residual_prediction = layers.Dense(1, name='residual_prediction')(x)

    # --- Final Output ---
    # The final prediction is the sum of the baseline and the specialist's correction
    final_prediction = layers.Add(name='final_prediction')([baseline_prediction, residual_prediction])
    
    model = tf.keras.Model(
        inputs=[sequence_input, scalar_input, stats_input], 
        outputs=final_prediction
    )
    return model

# --- 4. Visualization Function ---

def create_visualizations(results_df, output_dir='visualizations'):
    """Generates and saves plots comparing true vs. predicted total time."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    true_times = results_df['true_total_time']
    predicted_times = results_df['predicted_total_time']
    
    # --- Scatter Plot ---
    plt.figure(figsize=(10, 10))
    plt.scatter(true_times, predicted_times, alpha=0.6, label='Predictions')
    # Add a line for perfect prediction
    lims = [
        np.min([plt.xlim(), plt.ylim()]),
        np.max([plt.xlim(), plt.ylim()]),
    ]
    plt.plot(lims, lims, 'r--', alpha=0.75, zorder=0, label='Perfect Prediction')
    plt.xlabel("True Total Time (seconds)")
    plt.ylabel("Predicted Total Time (seconds)")
    plt.title("True vs. Predicted Total Time")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'true_vs_predicted_scatter.png'))
    plt.close()

    # --- Error Histogram ---
    errors = predicted_times - true_times
    plt.figure(figsize=(10, 6))
    plt.hist(errors, bins=30, alpha=0.7) # Increased bins for more detail
    plt.xlabel("Prediction Error (Predicted - True)")
    plt.ylabel("Frequency")
    plt.title("Distribution of Prediction Errors")
    plt.grid(True)
    plt.axvline(x=0, color='r', linestyle='--', linewidth=2)
    plt.savefig(os.path.join(output_dir, 'prediction_error_histogram.png'))
    plt.close()
    
    print(f"✅ Basic visualizations saved to '{output_dir}' directory.")

def create_advanced_visualizations(results_df, output_dir='visualizations'):
    """Generates and saves advanced diagnostic plots."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    true_times = results_df['true_total_time']
    predicted_times = results_df['predicted_total_time']
    residuals = true_times - predicted_times

    # --- Residuals vs. Predicted Plot ---
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=predicted_times, y=residuals, alpha=0.6)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel("Predicted Total Time (seconds)")
    plt.ylabel("Residuals (True - Predicted)")
    plt.title("Residuals vs. Predicted Values")
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'residuals_vs_predicted.png'))
    plt.close()

    # --- Predicted vs. True Distribution Plot ---
    plt.figure(figsize=(10, 6))
    sns.histplot(true_times, color="blue", label='True Values', kde=True, stat="density", linewidth=0)
    sns.histplot(predicted_times, color="red", label='Predicted Values', kde=True, stat="density", linewidth=0)
    plt.title("Distribution of Predicted vs. True Values")
    plt.xlabel("Total Time (seconds)")
    plt.legend()
    plt.savefig(os.path.join(output_dir, 'predicted_vs_true_distribution.png'))
    plt.close()
    
    print(f"✅ Advanced visualizations saved to '{output_dir}' directory.")


# --- 5. Training and Prediction Orchestration ---

def main():
    """Main function to run the data processing, training, and prediction."""
    
    proportions_file = 'prediction_176401_proportions_final_all.csv'
    ground_truth_file = 'data/176401/encoded_176401_condensed_full.csv'
    output_predictions_file = 'prediction_176401_total_time_full.csv'
    
    X_seq, X_steps, X_stats, y, props_df = load_and_preprocess_data(proportions_file, ground_truth_file)
    if X_seq is None:
        return

    # --- Prepare data for training with scaling ---
    X_seq_train, X_seq_val, X_steps_train, X_steps_val, X_stats_train, X_stats_val, y_train, y_val = train_test_split(
        X_seq, X_steps, X_stats, y, test_size=0.2, random_state=42
    )
    
    # --- Scaling Features ---
    scaler_steps = StandardScaler()
    X_steps_train_scaled = scaler_steps.fit_transform(X_steps_train)
    X_steps_val_scaled = scaler_steps.transform(X_steps_val)

    scaler_stats = StandardScaler()
    X_stats_train_scaled = scaler_stats.fit_transform(X_stats_train)
    X_stats_val_scaled = scaler_stats.transform(X_stats_val)
    
    scaler_y = StandardScaler()
    y_train_scaled = scaler_y.fit_transform(y_train)
    y_val_scaled = scaler_y.transform(y_val)
    
    # Define input shapes
    sequence_shape, scalar_shape, stats_shape = X_seq_train.shape[1:], (1,), X_stats_train.shape[1:]
    model = build_residual_attention_model(sequence_shape, scalar_shape, stats_shape)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.keras.losses.Huber(),
        metrics=['mae']
    )
    model.summary()
    
    print("\n--- Starting Model Training ---")
    model.fit(
        [X_seq_train, X_steps_train_scaled, X_stats_train_scaled],
        y_train_scaled,
        validation_data=([X_seq_val, X_steps_val_scaled, X_stats_val_scaled], y_val_scaled),
        epochs=500,
        batch_size=32,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)]
    )
    print("--- LSTM Model Training Finished ---\n")

    # --- Generate Predictions and Create Final Output ---
    print("--- Generating total time predictions for the entire dataset ---")
    X_steps_scaled = scaler_steps.transform(X_steps)
    X_stats_scaled = scaler_stats.transform(X_stats)
    scaled_predictions = model.predict([X_seq, X_steps_scaled, X_stats_scaled])
    
    predicted_times = scaler_y.inverse_transform(scaled_predictions).flatten()
    
    seq_order_to_time = dict(zip(props_df['SeqOrder'].unique(), predicted_times))
    
    props_df['predicted_total_time'] = np.nan
    end_marker_indices = props_df[props_df['sourceID'] == 10].groupby('SeqOrder')['Step'].idxmin()

    for seq_order, idx in end_marker_indices.items():
        if seq_order in seq_order_to_time:
            props_df.loc[idx, 'predicted_total_time'] = seq_order_to_time[seq_order]

    props_df.to_csv(output_predictions_file, index=False)
    print(f"✅ Final predictions with total time saved to '{output_predictions_file}'")

    # --- Create Results DataFrame for Visualization ---
    true_total_times_all = scaler_y.inverse_transform(y).flatten()
    results_df = pd.DataFrame({
        'SeqOrder': props_df['SeqOrder'].unique(),
        'true_total_time': true_total_times_all,
        'predicted_total_time': predicted_times
    })
    
    create_visualizations(results_df)
    create_advanced_visualizations(results_df)

    print("\n--- Sample of Final Predictions ---")
    # Display results for a sequence to verify the output format
    if not props_df.empty:
        first_seq_order = props_df['SeqOrder'].iloc[0]
        print(props_df[props_df['SeqOrder'] == first_seq_order])


In [36]:
if __name__ == "__main__":
    main()

Successfully processed 223 sequences.


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
